Подготовьте выборку: удалите столбцы `['id', 'date', 'price', 'zipcode']`, сформируйте обучающую и тестовую выборки по 10 тысяч домов.

Добавьте в тестовую выборку 10 новых объектов, в каждом из которых испорчен ровно один признак — например, это может быть дом из другого полушария, из далёкого прошлого или будущего, с площадью в целый штат или с таким числом этажей, что самолётам неплохо бы его облетать стороной.

Посмотрим на методы обнаружения аномалий на более простых данных — уж на табличном датасете с 19 признаками всё должно работать как надо!

Скачайте данные о стоимости домов: https://www.kaggle.com/harlfoxem/housesalesprediction/data

In [10]:
import numpy as np
import pandas as pd
import requests
import random
random.seed(10)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split

In [11]:
df = pd.read_csv('kc_house_data.csv').drop(['id', 'date', 'price', 'zipcode'], axis=1)
for i in range(10):
    df = pd.concat([df, pd.DataFrame([df.loc[random.randint(0, 21000)]])], ignore_index=True)
df['flag'] = [1 for i in range(21623)]
df.loc[21613,'bedrooms'] = 2000
df.loc[21614,'bathrooms'] = 100000
df.loc[21615,'sqft_living'] = 17075200000
df.loc[21616,'floors'] = 10000
df.loc[21617,'grade'] = 100
df.loc[21618,'yr_built'] = 0
df.loc[21619,'lat'] = 55.719317
df.loc[21619,'long'] = 37.464834
df.loc[21620,'condition'] = -1
df.loc[21621,'sqft_lot'] = 0
df.loc[21622,'floors'] = 0
for i in range(21613, 21623):
    df.loc[i,'flag'] = -1

In [12]:
df

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,flag
0,3.0,1.00,1180.0,5650.0,1.0,0.0,0.0,3.0,7.0,1180.0,0.0,1955.0,0.0,47.511200,-122.257000,1340.0,5650.0,1
1,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,3.0,7.0,2170.0,400.0,1951.0,1991.0,47.721000,-122.319000,1690.0,7639.0,1
2,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,3.0,6.0,770.0,0.0,1933.0,0.0,47.737900,-122.233000,2720.0,8062.0,1
3,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,5.0,7.0,1050.0,910.0,1965.0,0.0,47.520800,-122.393000,1360.0,5000.0,1
4,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,3.0,8.0,1680.0,0.0,1987.0,0.0,47.616800,-122.045000,1800.0,7503.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21618,6.0,3.50,4860.0,11793.0,2.0,0.0,0.0,3.0,11.0,3860.0,1000.0,0.0,0.0,47.552100,-122.137000,3600.0,11793.0,-1
21619,4.0,1.75,1660.0,10757.0,1.0,0.0,0.0,3.0,7.0,1000.0,660.0,1980.0,0.0,55.719317,37.464834,1880.0,8319.0,-1
21620,3.0,1.75,1720.0,9775.0,1.0,0.0,0.0,-1.0,8.0,1720.0,0.0,1978.0,0.0,47.373100,-122.286000,1970.0,8400.0,-1
21621,4.0,2.50,3030.0,0.0,2.0,0.0,0.0,3.0,9.0,1580.0,1450.0,1991.0,0.0,47.689000,-122.196000,2840.0,7435.0,-1


In [13]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('flag', axis=1), df.flag, test_size=10000, random_state=42, shuffle=False)

**Задание 9. (2 балла)**

Примените IsolationForest для поиска аномалий в этих данных, запишите их качество (как и раньше, это precision и recall). Проведите исследование:

Нарисуйте распределения всех признаков и обозначьте на этих распределениях объекты, которые признаны аномальными.

In [14]:
from sklearn.metrics import make_scorer, f1_score
from sklearn import model_selection
isolation_forest = IsolationForest(random_state=42, contamination=0.001)
param_grid = {'n_estimators': [50, 100, 150],
              'max_samples': ['auto', 0.1, 0.3, 0.5],
              'max_features': [5, 10, 17], 
              'contamination': [0.001],
              'n_jobs' : [-1],
              'bootstrap': [True, False]}

grid_dt_estimator.fit(X_train, y_train)
isolation_forest.fit(X_train)

test_predictions = isolation_forest.predict(X_test)

precision = precision_score(y_test, test_predictions, average='micro')
recall = recall_score(y_test, test_predictions, average='micro')
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


Precision: 0.9978
Recall: 0.9978


In [15]:
df

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,flag
0,3.0,1.00,1180.0,5650.0,1.0,0.0,0.0,3.0,7.0,1180.0,0.0,1955.0,0.0,47.511200,-122.257000,1340.0,5650.0,1
1,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,3.0,7.0,2170.0,400.0,1951.0,1991.0,47.721000,-122.319000,1690.0,7639.0,1
2,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,3.0,6.0,770.0,0.0,1933.0,0.0,47.737900,-122.233000,2720.0,8062.0,1
3,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,5.0,7.0,1050.0,910.0,1965.0,0.0,47.520800,-122.393000,1360.0,5000.0,1
4,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,3.0,8.0,1680.0,0.0,1987.0,0.0,47.616800,-122.045000,1800.0,7503.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21618,6.0,3.50,4860.0,11793.0,2.0,0.0,0.0,3.0,11.0,3860.0,1000.0,0.0,0.0,47.552100,-122.137000,3600.0,11793.0,-1
21619,4.0,1.75,1660.0,10757.0,1.0,0.0,0.0,3.0,7.0,1000.0,660.0,1980.0,0.0,55.719317,37.464834,1880.0,8319.0,-1
21620,3.0,1.75,1720.0,9775.0,1.0,0.0,0.0,-1.0,8.0,1720.0,0.0,1978.0,0.0,47.373100,-122.286000,1970.0,8400.0,-1
21621,4.0,2.50,3030.0,0.0,2.0,0.0,0.0,3.0,9.0,1580.0,1450.0,1991.0,0.0,47.689000,-122.196000,2840.0,7435.0,-1


In [16]:
df['anomaly'] = isolation_forest.predict(df.drop(['flag'], axis=1))
df

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,flag,anomaly
0,3.0,1.00,1180.0,5650.0,1.0,0.0,0.0,3.0,7.0,1180.0,0.0,1955.0,0.0,47.511200,-122.257000,1340.0,5650.0,1,1
1,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,3.0,7.0,2170.0,400.0,1951.0,1991.0,47.721000,-122.319000,1690.0,7639.0,1,1
2,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,3.0,6.0,770.0,0.0,1933.0,0.0,47.737900,-122.233000,2720.0,8062.0,1,1
3,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,5.0,7.0,1050.0,910.0,1965.0,0.0,47.520800,-122.393000,1360.0,5000.0,1,1
4,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,3.0,8.0,1680.0,0.0,1987.0,0.0,47.616800,-122.045000,1800.0,7503.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21618,6.0,3.50,4860.0,11793.0,2.0,0.0,0.0,3.0,11.0,3860.0,1000.0,0.0,0.0,47.552100,-122.137000,3600.0,11793.0,-1,1
21619,4.0,1.75,1660.0,10757.0,1.0,0.0,0.0,3.0,7.0,1000.0,660.0,1980.0,0.0,55.719317,37.464834,1880.0,8319.0,-1,1
21620,3.0,1.75,1720.0,9775.0,1.0,0.0,0.0,-1.0,8.0,1720.0,0.0,1978.0,0.0,47.373100,-122.286000,1970.0,8400.0,-1,1
21621,4.0,2.50,3030.0,0.0,2.0,0.0,0.0,3.0,9.0,1580.0,1450.0,1991.0,0.0,47.689000,-122.196000,2840.0,7435.0,-1,1


In [17]:
df[df.anomaly == -1]

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,flag,anomaly
527,6.0,5.00,6050.0,230652.0,2.0,0.0,3.0,3.0,11.0,6050.0,0.0,2001.0,0.0,47.6033,-121.943,4210.0,233971.0,1,-1
1164,5.0,5.25,8010.0,45517.0,2.0,1.0,4.0,3.0,12.0,5990.0,2020.0,1999.0,0.0,47.6767,-122.211,3430.0,26788.0,1,-1
1315,6.0,6.00,7390.0,24829.0,2.0,1.0,4.0,4.0,12.0,5000.0,2390.0,1991.0,0.0,47.5631,-122.210,4320.0,24619.0,1,-1
2626,5.0,5.50,6640.0,40014.0,2.0,1.0,4.0,3.0,12.0,6350.0,290.0,2004.0,0.0,47.7493,-122.280,3030.0,23408.0,1,-1
3914,5.0,4.50,10040.0,37325.0,2.0,1.0,2.0,3.0,11.0,7680.0,2360.0,1940.0,2001.0,47.6500,-122.214,3930.0,25449.0,1,-1
6501,4.0,5.75,7220.0,223462.0,2.0,0.0,4.0,3.0,12.0,6220.0,1000.0,2000.0,0.0,47.7097,-122.013,2680.0,7593.0,1,-1
6508,6.0,4.75,5480.0,19401.0,1.5,1.0,4.0,5.0,11.0,3910.0,1570.0,1936.0,0.0,47.6515,-122.277,3510.0,15810.0,1,-1
7252,6.0,8.00,12050.0,27600.0,2.5,0.0,3.0,4.0,13.0,8570.0,3480.0,1910.0,1987.0,47.6298,-122.323,3940.0,8800.0,1,-1
8092,5.0,6.75,9640.0,13068.0,1.0,1.0,4.0,3.0,12.0,4820.0,4820.0,1983.0,2009.0,47.5570,-122.210,3270.0,10454.0,1,-1
8277,3.0,2.50,2090.0,32718.0,2.0,1.0,4.0,3.0,7.0,1550.0,540.0,1919.0,1983.0,47.3338,-122.511,1200.0,192268.0,1,-1


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

features = df.columns[:-2] 
n_features = len(features)

plt.figure(figsize=(15, n_features * 3))

for i, feature in enumerate(features):
    plt.subplot(n_features, 1, i + 1)
    sns.histplot(df[feature], bins=50, kde=True, color='blue', label='normal')
    sns.histplot(df[df['anomaly'] == -1][feature], bins=50, kde=True, color='red', label='anomaly')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
df